In [ ]:
## Analyze TAVR scans' SNRs 

# part 1: pull and open TAVR scan - DONE

# part 2: click image on RV and myocardium to derive SNR - DONE

# part 3: save values and output as pandas dataframe or maybe as txt file ? - DONE

In [438]:
# part 1: pull and open TAVR scan

# inputs: folder name
# output: pop up of image slice

import os
import glob
import matplotlib
import math
import pandas as pd
import nibabel as nb

%matplotlib qt
import matplotlib.pyplot as plt
import numpy as np

TAVR_names = pd.read_excel('/Users/andersonscott/Desktop/TAVR_CT_Scans_CVC_Names_analyzed.xlsx')

patient_file_directory = '/Volumes/MainShare/projects/tavr_function'

patients = glob.glob(os.path.join(patient_file_directory,'*'))

TAVR_CNR = pd.read_excel('TAVR_CNR.xlsx')
patient_file = '/Volumes/MainShare/projects/tavr_function'


for TAVR_Name in TAVR_names['CVC Name']:
    if math.isnan(TAVR_names.loc[TAVR_names['CVC Name']==TAVR_Name, 'CNR']):
        print(TAVR_Name)
        first_img = os.path.join(patient_file,TAVR_Name+'/img-nii/0.nii.gz')
        if os.path.isfile(first_img):
            image_file = nb.load(first_img)

            image_data = image_file.get_fdata()

            slicez = round(image_data.shape[2]/2.5)
            ref_pts = get_pts(image_data,slicez)

            signal_pts = ref_pts[0]
            myo_pts = ref_pts[1]
            slicez = ref_pts[2]

            CNR = get_CNR(image_data,signal_pts,myo_pts,slicez)

            TAVR_names.loc[TAVR_names['CVC Name']==TAVR_Name, 'CNR'] = CNR

#TAVR_names.to_excel('TAVR_CT_Scans_CVC_Names_analyzed.xlsx', index = False)


CVC1505280957
CVC1506121406
CVC1706280900
CVC1708312050
CVC1709041420
CVC2001161230
CVC2005041159
CVC2006221423
CVC2007281019
CVC2008171525
CVC2008211440
CVC2008241337
CVC2009281439
CVC2010011447
CVC2010221507
CVC2010291455
CVC2011050951
CVC2011091349
CVC2011101418
CVC2011121458
CVC2011161512
CVC2011191311
CVC2011231021
CVC2011231437
CVC2012040057
CVC2012211016
CVC2012220907
CVC2012280929
CVC2101060808
CVC2101070919
CVC2101140921
CVC2101141509
CVC2101201006
CVC2102181027
CVC2103190853
CVC2104011440
CVC2104191500
CVC2105131001
CVC2106031542
CVC2106141023
CVC2106141429
CVC2107091202
CVC2107141518
CVC2107151449
CVC2107192159
CVC2107291421
CVC2107301320
CVC2108301640
CVC2110041316
CVC2110180952
CVC2110281449
CVC2201141200
CVC2202241037
CVC2207061507
CVC2207111037
CVC2207181018
CVC2207181518
CVC2208010911
CVC2208011048
CVC2208021509
CVC2208151003
CVC2208220913
CVC2209011020
CVC2209061436
CVC2209121057
CVC2209151310
CVC2209191003
CVC2209201107
CVC2209261456
CVC2210030934
CVC2210031106
CVC221

In [436]:
TAVR_names.loc[TAVR_names['CVC Name']==TAVR_Name, 'Function scan'][0]

False

In [306]:
# Cell where we save the functions

def get_pts(image,slicez):
    plt.close()

    while True:
           
        plt.imshow(image[:,:,slicez])

        
        plt.title('Click on an area of RV enhancement', fontsize=16)
        plt.draw()

        signal_pts = plt.ginput(1)[0]
        plt.scatter(signal_pts[0],signal_pts[1])


        plt.title('Click on an area of myocardial enhancement', fontsize=16)
        plt.draw()

        myo_pts = plt.ginput(1)[0]
        plt.scatter(myo_pts[0],myo_pts[1])

        plt.title('Happy? Key click for yes, mouse click for no',fontsize=16)
        plt.draw()

        if plt.waitforbuttonpress():
            break

        del pts
        plt.clf()
    
    plt.close()
    return signal_pts, myo_pts, slicez


def get_CNR(image_data,signal_pts,myo_pts,slicez,area_of_interest_radius = 8):
    
 #   plt.close()
    xref = int(signal_pts[0])
    yref = int(signal_pts[1])

    # Note: the indexing is backwards for x and y 
    signal1 = image_data[yref-area_of_interest_radius:yref+area_of_interest_radius,
                         xref-area_of_interest_radius:xref+area_of_interest_radius,
#                         slicez-area_of_interest_radius:slicez+area_of_interest_radius]
                         slicez]

    xref = int(myo_pts[0])
    yref = int(myo_pts[1])

    signal2 = image_data[yref-area_of_interest_radius:yref+area_of_interest_radius,
               xref-area_of_interest_radius:xref+area_of_interest_radius,
  #             slicez-area_of_interest_radius:slicez+area_of_interest_radius]
               slicez]

    CNR = abs(np.mean(signal1) - np.mean(signal2))/np.std(signal2)
    return CNR

def drawsquare(center,radius):
    centerx = center[0]
    centery = center[1]
    xpts = [centerx-radius,centerx+radius,centerx+radius,centerx-radius,centerx-radius]
    ypts = [centery+radius,centery+radius,centery-radius,centery-radius,centery+radius]
    plt.plot(xpts,ypts)



In [274]:
# View patients of varying SNRs
ex_pats = [
    'CVC1903111026',
    'CVC1911191729',
    'CVC1909091028',
    'CVC2001020922',
    'CVC2001231518',
    'CVC2001230926'
]

for i, ex_pat in enumerate(ex_pats):
    first_img = os.path.join(patient_file,ex_pat+'/img-nii/0.nii.gz')

    image_file = nb.load(first_img)

    image_data = image_file.get_fdata()

    slicez = round(image_data.shape[2]/2.5)
    plt.subplot(2,3,i+1)
    exslice = image_data[:,:,slicez]
    exslice[exslice <-1000] = -1000
    exslice[exslice > 1000] = 1000
    plt.imshow(exslice)
    CNR_img = round(TAVR_CNR.loc[TAVR_CNR["Name"]==ex_pat,'CNR'],2)
    plt.title(str(CNR_img))
plt.show()

In [328]:
plt.hist(TAVR_CNR['CNR'])
plt.show()

/usr/local/lib/python3.7/site-packages/numpy/lib/histograms.py:839: RuntimeWarning: invalid value encountered in greater_equal
  keep = (tmp_a >= first_edge)
/usr/local/lib/python3.7/site-packages/numpy/lib/histograms.py:840: RuntimeWarning: invalid value encountered in less_equal
  keep &= (tmp_a <= last_edge)


In [327]:
plt.figure(1)
plt.scatter(range(len(TAVR_CNR)),TAVR_CNR.sort_values(by='CNR')['CNR'])
plt.show()

In [366]:
need_to_move = []
for TAVR_name in TAVR_names['CVC Name']:
    if TAVR_names.loc[TAVR_names['CVC Name'] == TAVR_name, 'Function scan'].bool():
        if math.isnan(TAVR_CNR.loc[TAVR_CNR['Name'] == TAVR_name, 'CNR']):
#            print('Need to move '+TAVR_name+' files')
            need_to_move.append(TAVR_name)
            
need_to_move_df = pd.DataFrame({'CVC Name': need_to_move})

In [409]:
CNR_ex = []
CNR_ex.append((TAVR_CNR.CNR.values < 1).argmax())
CNR_ex.append((abs(TAVR_CNR.CNR.values - 2) < 1).argmax())
CNR_ex.append((abs(TAVR_CNR.CNR.values  - 3) < 1).argmax())
CNR_ex.append((abs(TAVR_CNR.CNR.values  - 5) < 1).argmax())
CNR_ex.append((abs(TAVR_CNR.CNR.values  - 8) < 1).argmax())
CNR_ex.append((abs(TAVR_CNR.CNR.values  - 10) < 1).argmax())


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in less
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in less
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in less
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in less
  """
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in less
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in less
  import sys


In [410]:
CNR_ex

[37, 1, 4, 3, 8, 12]

In [415]:
for i, ex_pat in enumerate(CNR_ex):
    ex_pat_name = TAVR_CNR['Name'][ex_pat]
    first_img = os.path.join(patient_file,ex_pat_name+'/img-nii/0.nii.gz')

    image_file = nb.load(first_img)

    image_data = image_file.get_fdata()

    slicez = round(image_data.shape[2]/2.5)
    plt.subplot(2,3,i+1)
    exslice = image_data[:,:,slicez]
    exslice[exslice <-1000] = -1000
    exslice[exslice > 1000] = 1000
    plt.imshow(exslice)
    CNR_img = round(TAVR_CNR['CNR'][ex_pat],2)
    plt.title(str(CNR_img))
plt.show()

In [416]:
len(TAVR_CNR)

208

In [417]:
len(TAVR_names)

246

In [422]:
new = TAVR_names.merge(TAVR_CNR.rename(columns = {'Name':'CVC Name'}), on='CVC Name', how='left')

In [426]:
new.to_excel('/Users/andersonscott/Desktop/TAVR_CT_Scans_CVC_Names_analyzed.xlsx')

In [421]:
TAVR_CNR

,Name,CNR
0,CVC1811052038,18.156579
1,CVC1903111026,1.513803
2,CVC1909091028,6.838564
3,CVC1910291016,5.671267
4,CVC1911071042,3.704908
...,...,...
203,CVC2106141429,NaN
204,CVC2108122137,12.040111
205,CVC2202241037,NaN
206,CVC2204260823,12.415319


In [439]:
TAVR_names

,Unnamed: 0,Study ID,CVC Name,Location,Aorta scan,Chest scan,Function scan,RR filenames,CNR
0,0,202,CVC1505280957,NaN,False,False,False,NaN,NaN
1,1,108,CVC1506121406,/Volumes/McVeighLab/wip/ucsd_tavr_2023_outcome...,False,False,False,NaN,NaN
2,2,221,CVC1706280900,NaN,False,False,False,NaN,NaN
3,3,221,CVC1708312050,NaN,False,False,False,NaN,NaN
4,4,91,CVC1709041420,NaN,False,False,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...
241,241,277,CVC2210312059,NaN,False,False,False,NaN,NaN
242,242,281,CVC2211041526,/Volumes/MainShare-1/dicom_images/2022/11/CVC2...,True,False,True,['/Volumes/MainShare-1/dicom_images/2022/11/CV...,17.727455
243,243,283,CVC2211301449,/Volumes/MainShare-1/dicom_images/2022/11/CVC2...,True,False,True,['/Volumes/MainShare-1/dicom_images/2022/11/CV...,5.073973
244,244,284,CVC2212131455,/Volumes/MainShare-1/dicom_images/2022/12/CVC2...,True,False,True,['/Volumes/MainShare-1/dicom_images/2022/12/CV...,10.308051


In [440]:
TAVR_names.to_excel('/Users/andersonscott/Desktop/TAVR_CT_Scans_CVC_Names_analyzed.xlsx', index = False)


In [441]:
import matplotlib.pyplot as plt
plt.hist(TAVR_names['CNR'])
plt.show()

/usr/local/lib/python3.7/site-packages/numpy/lib/histograms.py:839: RuntimeWarning: invalid value encountered in greater_equal
  keep = (tmp_a >= first_edge)
/usr/local/lib/python3.7/site-packages/numpy/lib/histograms.py:840: RuntimeWarning: invalid value encountered in less_equal
  keep &= (tmp_a <= last_edge)


In [495]:
plt.figure(2)

ok = TAVR_names.CNR.values
ok.sort()
nonanok = [ ok3 for ok3 in ok if not math.isnan(ok3)]

ok2 = [100*x/len(nonanok) for x in range(0,len(nonanok))]

plt.xlabel('% of Data')
plt.ylabel('CNR')

plt.plot(ok2,nonanok)
plt.show()

Text(94.06944444444443, 0.5, 'CNR')